In [ ]:
import requests
import json

base_url = "https://tigerweb.geo.census.gov/arcgis/rest/services/TIGERweb"
state_fips = {'MD':'24','SC':'45'}
county_params = "&outFields=NAME,COUNTY&f=geojson"
state_params = "&outFields=NAME&f=geojson"

counties_url = "State_County/MapServer/1/query?where=STATE='"
congressional_districts_url = "Legislative/MapServer/0/query?where=STATE='"
state_url = "State_County/MapServer/0/query?where=STATE='"

sc_counties_url = base_url + "/" + counties_url + state_fips['SC'] + "'" + county_params
md_counties_url = base_url + "/" + counties_url + state_fips['MD'] + "'" + county_params

sc_CD_url = base_url + "/" + congressional_districts_url + state_fips['SC'] + "'" + state_params
md_CD_url = base_url + "/" + congressional_districts_url + state_fips['MD'] + "'" + state_params

sc_state_url = base_url + "/" + state_url + state_fips['SC'] + "'" + state_params
md_state_url = base_url + "/" + state_url + state_fips['MD'] + "'" + state_params


In [ ]:
response = requests.get(sc_CD_url)
output_file  = "south_carolina_cd.geojson"
if response.status_code == 200:
    geojson_data = response.json()
    
    if "features" in geojson_data and len(geojson_data["features"]) > 0:
        names = [feature['properties']['NAME'] for feature in geojson_data['features']]
        print("SC CD:")
        for name in names:
            print(f"- {name}")
        with open(output_file, 'w') as file:
            json.dump(geojson_data, file, indent=4)
        print(f"GeoJSON data with county boundaries successfully saved to {output_file}")
    else:
        print("No county features found in the response. Please check the query parameters.")
else:
    print(f"Failed to retrieve data. HTTP Status Code: {response.status_code}")

SC CD:
- Congressional District 1
- Congressional District 2
- Congressional District 3
- Congressional District 4
- Congressional District 5
- Congressional District 6
- Congressional District 7
GeoJSON data with county boundaries successfully saved to south_carolina_cd.geojson


In [18]:
response = requests.get(md_CD_url)
output_file  = "maryland_cd.geojson"
if response.status_code == 200:
    geojson_data = response.json()
    
    if "features" in geojson_data and len(geojson_data["features"]) > 0:
        names = [feature['properties']['NAME'] for feature in geojson_data['features']]
        print("MD CD:")
        for name in names:
            print(f"- {name}")
        with open(output_file, 'w') as file:
            json.dump(geojson_data, file, indent=4)
        
        print(f"GeoJSON data with county boundaries successfully saved to {output_file}")
    else:
        print("No county features found in the response. Please check the query parameters.")
else:
    print(f"Failed to retrieve data. HTTP Status Code: {response.status_code}")

MD CD:
- Congressional District 1
- Congressional District 2
- Congressional District 3
- Congressional District 4
- Congressional District 5
- Congressional District 6
- Congressional District 7
- Congressional District 8
GeoJSON data with county boundaries successfully saved to maryland_cd.geojson


In [2]:
response = requests.get(md_state_url)
output_file  = "maryland_state.geojson"
if response.status_code == 200:
    geojson_data = response.json()
    
    if "features" in geojson_data and len(geojson_data["features"]) > 0:
        names = [feature['properties']['NAME'] for feature in geojson_data['features']]
        print("MD State:")
        for name in names:
            print(f"- {name}")
        with open(output_file, 'w') as file:
            json.dump(geojson_data, file, indent=4)
        
        print(f"GeoJSON data with county boundaries successfully saved to {output_file}")
    else:
        print("No county features found in the response. Please check the query parameters.")
else:
    print(f"Failed to retrieve data. HTTP Status Code: {response.status_code}")

MD State:
- Maryland
GeoJSON data with county boundaries successfully saved to maryland_state.geojson


In [3]:
response = requests.get(sc_state_url)
output_file  = "south_carolina_state.geojson"
if response.status_code == 200:
    geojson_data = response.json()
    
    if "features" in geojson_data and len(geojson_data["features"]) > 0:
        names = [feature['properties']['NAME'] for feature in geojson_data['features']]
        print("SC State:")
        for name in names:
            print(f"- {name}")
        with open(output_file, 'w') as file:
            json.dump(geojson_data, file, indent=4)
        
        print(f"GeoJSON data with county boundaries successfully saved to {output_file}")
    else:
        print("No county features found in the response. Please check the query parameters.")
else:
    print(f"Failed to retrieve data. HTTP Status Code: {response.status_code}")

SC State:
- South Carolina
GeoJSON data with county boundaries successfully saved to south_carolina_state.geojson
